In [0]:
import json, requests

def _ctx():
    c = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
    return {"host": c.apiUrl().get(), "token": c.apiToken().get()}

def _api(path, payload,method="get"):
    ctx = _ctx()
    url = f"{ctx['host']}{path}"
    headers = {"Authorization": f"Bearer {ctx['token']}",
               "Content-Type": "application/json"}
    if method == "get":
        r = requests.get(url, headers=headers)
        if r.status_code != 200:
            raise RuntimeError(f"{r.status_code}: {r.text}")
    else:
        r = requests.post(url, headers=headers, data=json.dumps(payload))
        if r.status_code != 200:
            raise RuntimeError(f"{r.status_code}: {r.text}")
    return r.json() if r.content else r


In [0]:
import secrets
salt_hex = secrets.token_hex(32)         # 32 byte → 64-char hex
print(salt_hex)                          # 例: e9ab0c... （毎回変わる）


シークレットの作成


In [0]:
request_path = r"/api/2.0/secrets/scopes/list"
_api(request_path,"",method="get")

In [0]:
_api("/api/2.0/secrets/scopes/create", {
    "scope": "mysalt02",
    "scope_backend_type": "DATABRICKS"
},method="post")


In [0]:
_api("/api/2.0/secrets/put", {
    "scope": "mysalt",
    "key":   "PASSWORD_SALT",
    "string_value": salt_hex
},method="post")


In [0]:
_api("/api/2.0/secrets/acls/put", {
    "scope": "mysalt",
    "principal": "admins",
    "permission": "MANAGE"
},method="post")


In [0]:
salt = dbutils.secrets.get(scope="mysalt", key="PASSWORD_SALT")


In [0]:
print(salt)

In [0]:
for character in salt:
    print(character)

In [0]:
# 保存データ
payload = {
    "secret_value": salt
}

# ファイル保存
with open("secret.enc", "w") as f:
    f.write(json.dumps(payload))

print("暗号化して secret.enc に保存しました。")